In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from time import time
import numpy as np
import plotly.graph_objects as go

from DEFAULTS import PARENT_PATH
import ap_field_jit as ap
import pan_mod_jit as pm
import tele_geo_jit as tg
import latrt_geo as lgeo

tele_geo_t = tg.initialize_telescope_geometry()

tg_th_1 =  tele_geo_t.th_1
tg_th2 = tele_geo_t.th2
tg_z_ap = tele_geo_t.z_ap
tg_th_fwhp = tele_geo_t.th_fwhp
tg_F_2 = tele_geo_t.F_2

tg_x_rotc = tele_geo_t.x_rotc
tg_y_rotc = tele_geo_t.y_rotc
tg_z_rotc = tele_geo_t.z_rotc

freq = 120.0 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01
k = 2 * np.pi / wavelength

dist = np.inf # set it infinitly far away
p_source = np.array([0, -7.2, dist])  # position of the source[m] 

tg_el0 = np.arctan(-(p_source[1] + 7.2) / p_source[2]) # elevation of telescope based on position of source tower [rad]
tg_az0 = np.arctan(-p_source[0] / p_source[2]) # azimuth of telescope based on position of source tower [rad]

error = 0 # mean mirror surface error [um]
adj1 = np.random.randn(1092) * error
adj2 = np.random.randn(1092) * error
save = 0
pan_mod2 = pm.panel_model_from_adjuster_offsets(
    2, adj2, 1, save
)  # Panel Model on M2
pan_mod1 = pm.panel_model_from_adjuster_offsets(
    1, adj1, 1, save
)  # Panel Model on M1

N_scan = 64
theta = np.linspace(-np.pi / 2 - 0.28, -np.pi / 2 + 0.28, N_scan)
phi = np.linspace(np.pi / 2 - 0.28, np.pi / 2 + 0.28, N_scan)

focus = np.array([0,0,0]) #the focus point

In [2]:
P_rx = focus # Receiver feed position in focal plane
rxmirror_t = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apfield_t = ap.aperature_fields_from_panel_model(pan_mod1, pan_mod2, \
                                        P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                        tg_F_2, theta, phi, rxmirror_t
                                        )

In [4]:
######### draw the mirrors and focal plane #############

miiror_size = 6000.0/np.cos(np.arctan(1 / 2))
msf = miiror_size/2.0

# positions of mirror 1 surface
x_m1 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
y_m1 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
x_m1, y_m1 = np.meshgrid(x_m1, y_m1)
height_1 = lgeo.z1(x_m1, y_m1)
x_m1, y_m1, z_m1 = lgeo.m1_into_tele(x_m1, y_m1, height_1)

# positions of mirror 2 surface
x_m2 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
y_m2 = np.linspace(-msf,msf,10, endpoint=True) # [mm]
x_m2, y_m2 = np.meshgrid(x_m2, y_m2)
height_2 = lgeo.z2(x_m2, y_m2)
x_m2, y_m2, z_m2 = lgeo.m2_into_tele(x_m2, y_m2, height_2)

# positions of focal plane 
x_fp = np.linspace(-2000,2000,100, endpoint=True) # [mm]
y_fp = np.linspace(-2000,2000,100, endpoint=True) # [mm]
x_fp, y_fp = np.meshgrid(x_fp, x_fp)
z_fp = lgeo.z_focal(x_fp,y_fp)
r = np.sqrt(x_fp**2 + y_fp**2)
z_fp = np.where(r<2000,z_fp,np.nan)
x_fp,y_fp,z_fp = lgeo.foc_into_tele(x_fp,y_fp,z_fp,tg_el0, tg_az0)

plot_m1 = go.Surface(
                            x = x_m1,
                            y = y_m1,
                            z = z_m1,
                            surfacecolor=height_1, 
                            colorscale='Teal',
                            showscale= False, 
                            name = 'Mirror 1',
                            text="Mirror 1", 
                            showlegend = True, 
                            hoverinfo="name"
                        )

plot_m2 = go.Surface(
                            x = x_m2,
                            y = y_m2,
                            z = z_m2,
                            surfacecolor=-height_2,
                            colorscale='Teal',
                            showscale= False, 
                            name = 'Mirror 2',
                            showlegend = True, 
                            hoverinfo="name"
                        )

plot_fp = go.Surface(
                            x = x_fp,
                            y = y_fp,
                            z = z_fp,
                            surfacecolor=z_fp*0 + 0.6,
                            colorscale='Cividis',
                            showscale= False, 
                            opacity=0.4,
                            name = 'Focal Plane',
                            showlegend = True, 
                            hoverinfo="name"
                        )

plot_setup = [plot_m1, plot_m2, plot_fp]


######### draw the rays obtained from the simulation #############
apfield_t = apfield_t
fp = np.where(apfield_t[15,:]!=0) #Ignore rays that missed the focal plane
amp_min = np.min(apfield_t[16,:][fp])
amp_max = np.max(apfield_t[16,:][fp])
x_foc, y_foc, z_foc = P_rx

plot_rays = []
x_a_list, y_a_list, z_a_list = [], [], []
amp_list = []
for ii in np.unique(np.random.randint(low=0, high=len(apfield_t[15]), size=1000)):
    if apfield_t[15][ii] !=0 :
        x_m1, y_m1, z_m1 = apfield_t[3][ii], apfield_t[4][ii], apfield_t[5][ii]
        x_m2, y_m2, z_m2 = apfield_t[0][ii], apfield_t[1][ii], apfield_t[2][ii]
        x_a, y_a, z_a = apfield_t[9][ii], apfield_t[10][ii], apfield_t[11][ii]


        xline = [ x_a, x_m1, x_m2, x_foc]
        yline = [ y_a, y_m1, y_m2, y_foc]
        zline = [ z_a, z_m1, z_m2, z_foc]
    
        amp = apfield_t[16,:][ii]

        my_color = ('rgba('+str(np.random.randint(50, high = 200))+','+
                    str(np.random.randint(50, high = 200))+','+
                    str(np.random.randint(50, high = 200)))

        ray_line =go.Scatter3d(
                            x=xline, 
                            y=yline, 
                            z=zline,
                            mode='lines',
                            line=dict(
                                    color=[amp,amp,amp,amp],
                                    cmax=amp_max, 
                                    cmin=amp_min,
                                    width=5, 
                                    colorscale='Purpor', 
                                    ), 
                            opacity=0.4,
                            showlegend = False,
                            hoverinfo='none', 
                            )

        plot_rays.append(ray_line)
        x_a_list.append(x_a)
        y_a_list.append(y_a)
        z_a_list.append(z_a)
        amp_list.append(amp)

plot_ap = [go.Scatter3d(
                        x = x_a_list, y = y_a_list, z = z_a_list, 
                        mode = 'markers', 
                        marker=dict(symbol="circle", size=5, colorscale='Purpor', color=amp_list), 
                        opacity=0.4,
                        showlegend = True,
                        hoverinfo='name', 
                        name="rays"
                      )
          ]

#################### show the plot #########################
plot_layout = go.Layout(title='LAT', autosize=False,
                  width=1000, height=600,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene = dict(
                                xaxis=dict(visible=False),
                                yaxis=dict(visible=False),
                                zaxis=dict(visible=False),
                                aspectmode='data'),
                )
fig = go.Figure(
                data=plot_setup + plot_rays + plot_ap, 
                layout=plot_layout
                )
fig.show()